In [2]:
from utils import convert_fasta_to_df, filter_df, add_source, make_balanced_df, exclude_same_seqs
import pandas as pd

#### Train p2

In [13]:
binders = convert_fasta_to_df("../data/uniprot/go_0003677_swissprot.fasta", mode=1, class_=1)

In [14]:
not_binders = convert_fasta_to_df("../data/uniprot/notgo_0003723_notgo_0003677_swissprot.fasta", mode=1, class_=0)

In [15]:
train_p2 = pd.concat([binders, not_binders])

In [16]:
train_p2 = filter_df(train_p2)

In [17]:
train_p2 = make_balanced_df(train_p2)

In [18]:
train_p2.to_csv("../data/embeddings/input_csv/train_p2.csv", index=False)

In [19]:
train_p2.label.value_counts()

1    34936
0    34936
Name: label, dtype: int64

#### Make trembl test data

Trembl annot score 5

In [10]:
binders = convert_fasta_to_df("../data/uniprot/uniprotkb_gene_AND_go_0003677_AND_revie_2024_02_29.fasta", mode=1, class_=1)

In [11]:
not_binders = convert_fasta_to_df("../data/uniprot/uniprotkb_gene_NOT_go_0003677_NOT_go_00_2024_02_29.fasta", mode=1, class_=0)

In [12]:
test = pd.concat([binders, not_binders])

In [13]:
test = filter_df(test)

In [14]:
test.label.value_counts()

0    115917
1      5097
Name: label, dtype: int64

In [15]:
test = make_balanced_df(test)

In [16]:
test.label.value_counts()

1    5097
0    5097
Name: label, dtype: int64

In [17]:
test.to_csv("../data/embeddings/input_csv/trembl_test.csv", index=False)

Exclude common seqs with my train

In [27]:
test = exclude_same_seqs(train_p2, test)

Exclude common seqs with proteinfer train

In [66]:
proteinfer_train = pd.read_csv("../data/proteinfer/proteinfer_random_split.csv")

In [33]:
test = exclude_same_seqs(proteinfer_train, test)

In [34]:
test = filter_df(test)

In [35]:
test = make_balanced_df(test)

In [42]:
test.label.value_counts()

1    4938
0    4938
Name: label, dtype: int64

In [1]:
# test.to_csv("../data/ready_data/test.csv", index=False)

In [50]:
test.head()

,identifier,label,sequence
1,A0A061HLT0,1,MQTDPPAGVSASPVADNVMIWNAVIIGPADTPFEDGTFRLVMQFEE...
2,A0A061IE76,1,MDNMSITNTPTSNDACLSIVHSLMCHRQGGESETFAKRAIESLVKK...
3,A0A061IIX2,1,MTAEQRQNLQAFRDYVRKTLDPTYILSYMSPWLQDDEVQYIQAEKN...
6,A0A075AP15,1,MTVPSTSHSAKPPSRPSLVPKKAEKPQLPSVPLHLVDKRDNDIQYI...
7,A0A075F5C6,1,MDLAVGPGAAGPSNVPAFLTKLWTLVSDPDTDALICWSPSGNSFHV...


In [51]:
test.sequence.apply(lambda x: len(x)).max()

1024

In [52]:
test.sequence.apply(lambda x: len(x)).min()

57

In [53]:
proteinfer_train.sequence.apply(lambda x: len(x)).min()

1

In [58]:
#proteinfer_train[proteinfer_train.sequence.apply(lambda x: 1 >= len(x))]

Исключить учитывая длину уже полученного теста

In [67]:
def valid_sequence(sequence: str) -> bool:
    valid_amino_acids = "SNYLRQDPMFCEWGTKIVAH"
    return all(char in valid_amino_acids for char in sequence)

In [68]:
proteinfer_train = proteinfer_train.loc[proteinfer_train["sequence"].apply(valid_sequence)]

In [69]:
proteinfer_train = proteinfer_train.loc[proteinfer_train["sequence"].apply(lambda x: 14 < len(x) < 2049)]

In [70]:
proteinfer_train["identifier"] = proteinfer_train["identifier"].astype(str)

In [71]:
proteinfer_train.to_csv("../data/proteinfer/proteinfer_train_2.csv", index=False)

In [74]:
proteinfer_train.shape

(412859, 2)

In [72]:
proteinfer_train.dtypes

identifier    object
sequence      object
dtype: object

#### Exclude seqs from test

In [107]:
test = pd.read_csv("../data/ready_data/test.csv")

In [108]:
def filter_test(clusters_train_test, train, test):
    a = clusters_train_test.merge(train, on=["identifier"])
    b = clusters_train_test.merge(test, on=["identifier"])

    exclude_test = a.merge(b, on=["cluster"])["identifier_y"].drop_duplicates()
    filtered_test = test.loc[~test["identifier"].isin(exclude_test)]
    return filtered_test
    

In [109]:
proteinfer_train = pd.read_csv("../data/proteinfer/proteinfer_train_2.csv")
proteinfer_train["identifier"] = proteinfer_train["identifier"].astype(str)

In [110]:
test_proteinfer = pd.read_csv("../data/ready_data/test_proteinfer_train_id0.25.csv")


In [111]:
test_1 = filter_test(test_proteinfer, proteinfer_train, test)

In [112]:
test_train_p2 = pd.read_csv("../data/ready_data/test_train_p2_id0.25.csv")
train_p2 = pd.read_csv("../data/embeddings/input_csv/train_p2.csv")

In [113]:
test_2 = filter_test(test_train_p2, train_p2, test)

In [114]:
idx = set(test_1["identifier"]).intersection(set(test_2["identifier"]))

In [115]:
filtered_test = test.loc[test["identifier"].isin(idx)]

In [116]:
filtered_test.label.value_counts()

1    82
0    13
Name: label, dtype: int64

In [117]:
filtered_test = make_balanced_df(filtered_test)

In [1]:
# filtered_test.label.value_counts()
# filtered_test.to_csv("")

#### Another test data from paper https://academic.oup.com/nar/article/43/22/e158/1805680#83573870

In [2]:
with open("../data/DBP_papers_data/arabidopsis_yest.fasta") as fi:
    lines = fi.readlines()

lines = [line.strip() for line in lines if line.strip()]

In [3]:
splits = []
a = 1
for i in range(1, len(lines)):
    if "DNA-BPs***" in lines[i]:
        splits.append((a, i))
        a = i + 1
        
splits.append((a, i + 1))

In [4]:
splits

[(1, 5416),
 (5417, 23601),
 (23602, 26044),
 (26045, 42753),
 (42754, 43334),
 (43335, 44367)]

In [5]:
def parse_chunk(coords, label=0):
    a, b = coords
    chunk = lines[a:b]
    
    identifiers = []
    sequences = []
    seq = ""
    for line in chunk:
        if ">" in line:
            identifiers.append(line[1:10].replace(" ", ""))
            if seq:
                sequences.append(seq)
            seq = ""
        else:
            seq += line

    if seq:
        sequences.append(seq)
    assert len(sequences) == len(identifiers)

    labels = [label] * len(identifiers)

    identifiers = [str(i) + identifiers[i] for i in range(len(identifiers))]

    df = pd.DataFrame({"identifier": identifiers, "label": labels,
                       "sequence": sequences})
    
    return df

In [6]:
df_1 = parse_chunk(splits[0], label=1)

In [7]:
df_2 = parse_chunk(splits[1], label=0)

In [8]:
yeast_test = pd.concat([df_1, df_2])

In [13]:
# yeast_test.to_csv("../data/ready_data/yeast.csv", index=False)

In [10]:
df_1 = parse_chunk(splits[2], label=1)

In [11]:
df_2 = parse_chunk(splits[3], label=0)

In [12]:
arabidopsis_test = pd.concat([df_1, df_2])

In [13]:
# arabidopsis_df.to_csv("../data/ready_data/arabidopsis.csv", index=False)

In [17]:
df_1 = parse_chunk(splits[4], label=1)
df_2 = parse_chunk(splits[5], label=0)

In [18]:
other_plant_df = pd.concat([df_1, df_2])

In [14]:
# other_plant_df.to_csv("../data/ready_data/other_plant.csv", index=False)

In [9]:
train = pd.read_csv("../data/embeddings/input_csv/train_p2.csv")

In [10]:
yeast_train = pd.read_csv("../data/ready_data/yeast_train_0.5.csv")

In [11]:
train = filter_train(yeast_train, train, yeast_test)

In [12]:
train

,identifier,cluster,label,sequence
0,Q9I9E1,1,1,MTKHWGGPGLLAPPVITVGEGAQRDHHLDCRIGYSSSKRSCHRSSN...
1,Q9IAC6,2,1,FKRAVEGQHNYLCAGRNDCIIDKIRRKNCPACRVRKCLQAGMNLGA...
2,Q28590,2,1,EASQSPQYSFESLPQKICLICGDEASGCHYGVLTCGSCKVFFKRAM...
3,Q9IZT2,3,1,MGVSGSKGQKLFVSVLQRLLSERGLHVKESSAIEFYQFLIKVSPWF...
4,P10258,3,1,MGVSGSKGQKLFVSVLQRLLSERGLHVKESSAIEFYQFLIKVSPWF...
...,...,...,...,...
66730,P37552,28895,0,MTKAVHTKHAPAAIGPYSQGIIVNNMFYSSGQIPLTPSGEMVNGDI...
66731,P0AF93,28895,0,MSKTIATENAPAAIGPYVQGVDLGNMIITSGQIPVNPKTGEVPADV...
66732,P08244,28896,0,MTLTASSSSRAVTNSPVVVALDYHNRDDALAFVDKIDPRDCRLKVG...
66733,O43684,28897,0,MTGSNEFKLNQPPEDGISSVKFSPNTSQFLLVSSWDTSVRLYDVPA...


In [36]:
train

,identifier,label,sequence
0,A0A096MJY4,1,MGRKKIQITRIMDERNRQVTFTKRKFGLMKKAYELSVLCDCEIALI...
1,A0A0D2UG83,1,MAKSKKIVAATSGSRSRSSRAGLAFPVGRVHRLLRKGHFADRIGSG...
2,A0A0G2JTZ2,1,MSSKQATSPFACTVDGEETMTQDLTSREKEEGSDQHPASHLPLHPI...
3,A0A0G2JZ79,1,MIGTDPRTILKDLLPETIPPPELDDMTLWQIVINILSEPPKRKKRK...
4,A0A0G2L7I0,1,MMEDEDFLLALRLQEQFDQETPAAGWPDEDCPSSKRRRVDPSGGLD...
...,...,...,...
69867,Q9Z0F8,0,MRRRLLILTTLVPFVLAPRPPEEAGSGSHPRLEKLDSLLSDYDILS...
69868,Q9LMX7,0,MSPEAYVLFFNSFNLVTFEAFASVSLIIATVAFLLSPGGLAWAWTG...
69869,Q63690,0,MDGSGEQLGGGGPTSSEQIMKTGAFLLQGFIQDRAGRMAGETPELT...
69870,P59823,0,MKAPIPHLILLYATFTQSLKVVTKRGSADGCTDWSVDIKKYQVLVG...
